In [1]:
import os
import cv2
import yaml
import pydicom
import numpy as np
import pandas as pd
import plistlib
from tqdm import tqdm

# ✅ Define dataset paths
dataset_root = "E:\PFE\Flower code\data created\client_inbreast_1"
# ALLDICOMs = "E:\PFE\Flower code\data original\INbreast Release 1.0\AllDICOMs"
# ALLXML = "E:\PFE\Flower code\data original\INbreast Release 1.0\AllXML"
# XLS_PATH = "E:\PFE\Flower code\data original\INbreast Release 1.0\INbreast.xls"

# # ✅ Create YOLO dataset structure
# train_img_dir = os.path.join(dataset_root, "train/images")
# train_lbl_dir = os.path.join(dataset_root, "train/labels")
# valid_img_dir = os.path.join(dataset_root, "valid/images")
# valid_lbl_dir = os.path.join(dataset_root, "valid/labels")
# os.makedirs(train_img_dir, exist_ok=True)
# os.makedirs(train_lbl_dir, exist_ok=True)
# os.makedirs(valid_img_dir, exist_ok=True)
# os.makedirs(valid_lbl_dir, exist_ok=True)

# # ✅ Load dataset metadata
# df = pd.read_excel(XLS_PATH)
# df.columns = df.columns.str.strip()
# df = df.dropna(subset=['File Name'])

# # ✅ Filter dataset for calcifications only
# birads_followup = ["4", "4a", "4b", "4c", "4d", "5", "6", "49", "2", "3"]
# df = df[(df['Bi-Rads'].astype(str) == "1") | (df['Bi-Rads'].astype(str).isin(birads_followup)) & (df["Mass"] == "X")].reset_index(drop=True)

# # ✅ Add binary label
# def map_label(bi_rads):
#     return 0 if str(bi_rads).strip() == "1" else 1
# df['label'] = df['Bi-Rads'].apply(map_label)

# # ✅ Manual stratified split
# train_df = pd.DataFrame()
# valid_df = pd.DataFrame()
# for label in df["label"].unique():
#     group = df[df["label"] == label].sample(frac=1, random_state=42).reset_index(drop=True)
#     split_idx = int(len(group) * 0.8)
#     train_df = pd.concat([train_df, group.iloc[:split_idx]])
#     valid_df = pd.concat([valid_df, group.iloc[split_idx:]])

# train_df = train_df.reset_index(drop=True)
# valid_df = valid_df.reset_index(drop=True)

# # ✅ Multiply after stratified split
# train_df = pd.concat([
#     pd.concat([g]*2 if k == 0 else [g]*7)
#     for k, g in train_df.groupby('label')
# ], ignore_index=True)

# valid_df = pd.concat([
#     pd.concat([g]*2 if k == 0 else [g]*7)
#     for k, g in valid_df.groupby('label')
# ], ignore_index=True)

# train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
# valid_df = valid_df.sample(frac=1, random_state=42).reset_index(drop=True)

# # ✅ Confirm label balance
# print("Train label counts:\n", train_df['label'].value_counts())
# print("Validation label counts:\n", valid_df['label'].value_counts())

# # ✅ Function to load calcification ROIs from XML
# def load_calcifications(xml_path, imshape):
#     def load_point(point_string):
#         x, y = tuple(map(float, point_string.strip('()').split(',')))
#         return y, x  # Convert to (row, col) format

#     boxes = []
#     with open(xml_path, 'rb') as mask_file:
#         plist_dict = plistlib.load(mask_file, fmt=plistlib.FMT_XML)['Images'][0]
#         for roi in plist_dict['ROIs']:
#             if "Mass" not in roi['Name']:
#                 continue
#             points = [load_point(pt) for pt in roi['Point_px']]
#             x, y = zip(*points)
#             x1, y1, x2, y2 = min(y), min(x), max(y), max(x)
#             width, height = x2 - x1, y2 - y1
#             if width > 0 and height > 0:
#                 boxes.append((x1, y1, x2, y2))
#     return boxes

# # ✅ Function to process and save images/labels
# def process_and_save(df, img_dir, lbl_dir):
#     for _, row in tqdm(df.iterrows(), total=len(df)):
#         file_name = str(int(row['File Name']))
#         birads = str(row['Bi-Rads']).strip()
#         label = 0 if birads == "1" else 1

#         dicom_file = next((f for f in os.listdir(ALLDICOMs) if f.startswith(file_name)), None)
#         if not dicom_file:
#             continue

#         dcm_path = os.path.join(ALLDICOMs, dicom_file)
#         image = pydicom.dcmread(dcm_path).pixel_array.astype(np.float32)
#         image = (image - image.min()) / (image.max() - image.min())
#         image = (image * 255).astype(np.uint8)

#         clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
#         image = clahe.apply(image)
#         image = np.stack([image, image, image], axis=-1)

#         img_filename = f"{file_name}.png"
#         img_path = os.path.join(img_dir, img_filename)
#         cv2.imwrite(img_path, image)

#         xml_file = next((f for f in os.listdir(ALLXML) if f.startswith(file_name)), None)
#         lbl_filename = f"{file_name}.txt"
#         lbl_path = os.path.join(lbl_dir, lbl_filename)

#         if label == 0:
#             open(lbl_path, "w").close()
#             continue

#         if not xml_file:
#             continue

#         xml_path = os.path.join(ALLXML, xml_file)
#         boxes = load_calcifications(xml_path, image.shape[:2])
#         h, w = image.shape[:2]

#         yolo_boxes = []
#         for (x1, y1, x2, y2) in boxes:
#             x_center = (x1 + x2) / 2 / w
#             y_center = (y1 + y2) / 2 / h
#             width = (x2 - x1) / w
#             height = (y2 - y1) / h
#             yolo_boxes.append(f"{label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

#         with open(lbl_path, "w") as f:
#             f.write("\n".join(yolo_boxes))

# # ✅ Process datasets
# process_and_save(train_df, train_img_dir, train_lbl_dir)
# process_and_save(valid_df, valid_img_dir, valid_lbl_dir)

# # ✅ YAML file
# yolo_config = {
#     "path": dataset_root,
#     "train": "train/images",
#     "val": "valid/images",
#     "names": {0: "no_mass", 1: "mass"}
# }

# with open(os.path.join(dataset_root, "inbreast.yaml"), "w") as yaml_file:
#     yaml.dump(yolo_config, yaml_file)

# print("✅ YOLO dataset for calcifications with stratified split and data multiplication created successfully!")


In [2]:
from ultralytics import YOLO

# Define the path to your YAML config file
data_yaml = os.path.join(dataset_root, "inbreast.yaml")

# Load YOLOv11 model (Choose a variant: n, s, m, l, x)
model = YOLO("E:\PFE\Flower code\yolo models\yolo11m.pt")  # Smallest model, change to 'yolov11m.pt' or 'yolov11l.pt' for bigger models
model.train(
    augment=True,
    data=data_yaml,  # Path to dataset YAML
    epochs=170,       # Number of training epochs
    imgsz=640,       # Image size for training
    batch=15,         # Batch size (adjust based on GPU memory)
    device="cuda", # Use GPU if available, else "cpu"
    workers=10,       # Number of CPU workers for data loading
    verbose=True,    # Print training logs
)


Ultralytics 8.3.107  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=E:\PFE\Flower code\yolo models\yolo11m.pt, data=E:\PFE\Flower code\data created\client_inbreast_1\inbreast.yaml, epochs=170, time=None, patience=100, batch=15, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=10, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_co

train: Scanning E:\PFE\Flower code\data created\client_inbreast_1\train\labels.cache... 139 images, 55 backgrounds, 0 corrupt: 100%|██████████| 139/139 [00:00<?, ?it/s]
val: Scanning E:\PFE\Flower code\data created\client_inbreast_1\valid\labels.cache... 36 images, 14 backgrounds, 0 corrupt: 100%|██████████| 36/36 [00:00<?, ?it/s]


Plotting labels to runs\detect\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.00046875), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 6 dataloader workers
Logging results to runs\detect\train10
Starting training for 170 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/10 [01:37<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 94.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 11.98 GiB is allocated by PyTorch, and 155.72 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
metrics = model.val(data=data_yaml, split="val")

# Print key evaluation metrics
print("Validation Results:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")


Ultralytics 8.3.100 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning /kaggle/working/inbreast_yolo_dataset/valid/labels.cache... 36 images, 14 backgrounds, 0 corrupt: 100%|██████████| 36/36 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.24it/s]


                   all         36         22      0.874      0.949      0.917      0.553
                  mass         22         22      0.874      0.949      0.917      0.553


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 5.0ms preprocess, 40.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train2
Validation Results:
mAP50: 0.9172
mAP50-95: 0.5531
Precision: 0.8744
Recall: 0.9493


In [ ]:
metrics = model.val(data=data_yaml, split="train")

# Print key evaluation metrics
print("Training Results:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

Ultralytics 8.3.100 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/inbreast_yolo_dataset/train/labels.cache... 139 images, 55 backgrounds, 0 corrupt: 100%|██████████| 139/139 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:10<00:00,  1.68it/s]


                   all        139         93      0.978      0.978      0.984      0.795
                  mass         84         93      0.978      0.978      0.984      0.795


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 2.4ms preprocess, 27.3ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/train3
Training Results:
mAP50: 0.9837
mAP50-95: 0.7955
Precision: 0.9776
Recall: 0.9785


In [ ]:
# import cv2
# import pydicom
# import numpy as np
# import matplotlib.pyplot as plt

# # Load DICOM image
# dcm_path = "/kaggle/input/inbreast2012/INbreast Release 1.0/AllDICOMs/22580098_6200187f3f1ccc18_MG_L_ML_ANON.dcm"
# dcm = pydicom.dcmread(dcm_path)
# image = dcm.pixel_array.astype(np.float32)

# # ✅ Normalize to [0, 255] and convert to uint8
# image_norm = (image - image.min()) / (image.max() - image.min())
# image_norm = (image_norm * 255).astype(np.uint8)

# # ✅ Apply CLAHE
# clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
# clahe_image = clahe.apply(image_norm)

# # ✅ Plot original and CLAHE-enhanced images side by side
# plt.figure(figsize=(12, 6))

# plt.subplot(1, 2, 1)
# plt.imshow(image_norm, cmap='gray')
# plt.title("Original (Normalized)")
# plt.axis("off")

# plt.subplot(1, 2, 2)
# plt.imshow(clahe_image, cmap='gray')
# plt.title("CLAHE Enhanced")
# plt.axis("off")

# plt.tight_layout()
# plt.show()
